# Interacting with a bitcoind node using Python
This guide assumes you have a bitcoind node running and available at `localhost:8332`. If not you should refer to this repository's `README.md` for how to set that up.

Requires:
```
python-bitcoinrpc (1.0)
```

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import json

Let's set the RPC user and password so that we can instantiate a connection to bitcoind:

In [ ]:
rpc_user = 'rpcuser'
rpc_password = 'rpcsecret'

Now we can go ahead and connect to the bitcoind node:

In [ ]:
rpc_conn = AuthServiceProxy("http://%s:%s@localhost:8332"%(rpc_user, rpc_password), timeout=10000)

Let's test the connection:

In [ ]:
rpc_conn.getinfo()

Note that the latest block at the time of writing was `498016`. You'll notice that if you just started up your bitcoind node it probably has far fewer blocks than that. This is because it is still in the process of downloading the full blockchain and getting up to speed. 

One thing to consider is that only blocks found on disk can be accessed using the RPC connection. So if you want to inspect a block you can, just ensure you are passing the hash of an early block. For example:

In [ ]:
rpc_conn.getblock('00000000000005cbac4729863520767efc50c505fe2743d5d8abb016683388e3')

Now we can define some helper functions to get us additional information. We will always pass the RPC connection (`rpc_conn`) to each function to remind us that we need a live connection in case it has timed out.

In [ ]:
def get_block_hash(rpc_conn, index):
    return rpc_conn.getblockhash(index)

def get_block(rpc_conn, hash):
    return rpc_conn.getblock(hash)

def get_raw_transaction(rpc_conn, tx):
    out = rpc_conn.getrawtransaction(tx, 1)
    return out

def get_tx_outputs(tx):
    outputs = [float(i['value']) for i in tx['vout']]
    return outputs

Now we can get a block using the block height as opposed to the block's raw hash:

In [ ]:
block = get_block(rpc_conn,get_block_hash(rpc_conn, 201286))

In [ ]:
block

The block is a dictionary which has a key called `tx` which lists each transaction included in the block. We can grab one of these transactions and inspect it further:

In [ ]:
sample_tx_hash = block['tx'][1]
tx = get_raw_transaction(rpc_conn, sample_tx_hash)
tx

Here we have the raw transaction. This contains all the information for this transaction including where the transaction originated, where it went, and its corresponding amounts.

By looking at the difference between the output from a transaction and its inputs we can determine the actual mining fees paid by the transactor:

In [ ]:
def get_tx_fee(rpc, tx):
    tx_inputs = 0
    tx_outputs = 0
    for vin in tx['vin']:
        tx_id = vin['txid']
        ind = vin['vout']
        inner_tx = get_raw_transaction(rpc, tx_id)
        inner_val = inner_tx['vout'][ind]['value']
        tx_inputs += inner_val
    for vout in tx['vout']:
        tx_outputs += vout['value']
    return tx_inputs - tx_outputs

In [ ]:
get_tx_fee(rpc_conn, tx)

In [ ]:
block = get_block(rpc_conn,rpc_conn.getbestblockhash())

We can also batch process blocks:

In [ ]:
commands = [ [ "getblockhash", height] for height in range(10) ]
block_hashes = rpc_conn.batch_(commands)
blocks = rpc_conn.batch_([ [ "getblock", h ] for h in block_hashes ])
block_times = [ block["time"] for block in blocks ]

This returns the epoch times for each block:

In [ ]:
block_times